### Import relevant packages, establish connection to WRDS and set overall configurations for the notebook

WRDS Support - https://wrds-www.wharton.upenn.edu/pages/support/programming-wrds/programming-python/querying-wrds-data-python/

In [ ]:
# Import packages
import os
import numpy as np
import pandas as pd
import wrds
import yfinance as yf 

# Build WRDS connection

db = wrds.Connection(wrds_username='tomasromeiro')
#db.close()

# Set option to display all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Set option to force dataframes to display numbers as floats with thousands separators
pd.set_option('display.float_format', '{:,.2f}'.format)  # Adjust decimal places as needed

Loading library list...
Done


WRDS Quick commands

In [ ]:
# List libraries available
sorted(db.list_libraries()) 

# List tables within a library
db.list_tables(library="cboe") 

# describe table metadata
db.describe_table(library="cboe", table="optprice_2024") 

# Execute a sql query against a table (join queries between tables in library can also be performed)
data = db.raw_sql('SELECT date, dji FROM djones.djdaily LIMIT 1', date_cols=['date']) 

# Pass parameters to a sql statement
params = {"tickers": ("0015B", "0030B", "0032A", "0033A", "0038A")}
data = db.raw_sql(
    "SELECT datadate, gvkey, cusip FROM comp.funda WHERE tic IN %(tickers)s LIMIT 1",
    params=params,
)

### 1. FINRA Short Interest Bimonthly Data  
- https://www.finra.org/finra-data/browse-catalog/equity-short-interest/files
- https://www.finra.org/finra-data/browse-catalog/equity-short-interest/glossary

#### a) Collate semi monthly datasets
If the data has already been collated do not run this and skip to b)

In [ ]:
# Define the directory containing the CSV files
directory = 'data/finra_short_interest_data'
output_file = os.path.join(directory, 'collated_short_interest_data.csv')

# Check if the collated file already exists and delete it
if os.path.exists(output_file):
    os.remove(output_file)

# Get a list of all pipe-delimited CSV files in the directory
csv_files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.csv')]

# Read and concatenate all CSV files with proper delimiter handling
df_list = []
for file in csv_files:
    try:
        df = pd.read_csv(file, sep='|')  # Read as pipe-delimited with specific dtype
        df_list.append(df)
    except Exception as e:
        print(f"Error reading {file}: {e}")

if df_list:
    short_interest_df = pd.concat(df_list, ignore_index=True)
    
    # Replace daysToCoverQuantity with blank where averageDailyVolumeQuantity is 0
    short_interest_df.loc[short_interest_df['averageDailyVolumeQuantity'] == 0, 'daysToCoverQuantity'] = None
    
    # Remove all entries where the ticker (symbolCode) is missing and daysToCoverQuantity is NaN or 999.99 
    short_interest_df = short_interest_df.dropna(subset=['symbolCode', 'daysToCoverQuantity'])
    short_interest_df = short_interest_df[short_interest_df['daysToCoverQuantity'] != 999.99]

    # Keep only stocks not traded Over the Counter
    short_interest_df = short_interest_df[short_interest_df['marketClassCode'] != 'OTC']

    # Drop unnecessary fields
    short_interest_df = short_interest_df.drop(columns=['accountingYearMonthNumber', 'issuerServicesGroupExchangeCode', 'stockSplitFlag', 'revisionFlag', 'changePercent', 'changePreviousNumber', 'previousShortPositionQuantity', 'issueName', 'marketClassCode'])
    
    # Move settlementDate to the first column
    columns = ['settlementDate'] + [col for col in short_interest_df.columns if col != 'settlementDate']
    short_interest_df = short_interest_df[columns]

    # Sort by settlementDate and symbolCode
    short_interest_df = short_interest_df.sort_values(by=['settlementDate', 'symbolCode'])

    # Renaming columns
    short_interest_df.rename(columns={"currentShortPositionQuantity": "short_volume"}, inplace=True)
    short_interest_df.rename(columns={"averageDailyVolumeQuantity": "avg_daily_volume"}, inplace=True)
    short_interest_df.rename(columns={"daysToCoverQuantity": "days_to_cover"}, inplace=True)

    # Chaging fields to appropriate data type
    short_interest_df = short_interest_df.astype({"short_volume": "int32", "avg_daily_volume": "int32"})

    # Save the collated DataFrame to the same directory
    short_interest_df.to_csv(output_file, index=False)
    
    print(f"Collated data saved to {output_file}")
else:
    print("No valid CSV files found.")

#### b) Open .csv file to memory

In [24]:
# Define the directory containing the CSV files
directory = 'data/finra_short_interest_data'

# Open the collated file in a DataFrame for viewing
short_interest_file = os.path.join(directory, 'collated_short_interest_data.csv')
short_interest_df = pd.read_csv(short_interest_file)   

Extract tickers and date ranges to use as parameters for remaining data extracts

In [25]:
# Extract unique tickers from the short interest file. Will be used as the main variable to pass through to subsquent queries where tickers are required
tickers = short_interest_df['symbolCode'].unique().tolist()
print(f"{len(tickers)} unique tickers in the short interest file")

# Extract earliest and latest date in the short interest file
earliest_date = short_interest_df['settlementDate'].min()
latest_date = short_interest_df['settlementDate'].max()

print(f"Short interest file date range is {earliest_date} to {latest_date}")

16857 unique tickers in the short interest file
Short interest file date range is 2021-06-15 to 2025-01-15


### 2. WRDS (Wharton) Data

#### i) Company Data (Quarterly) - Fundamentals
https://wrds-www.wharton.upenn.edu/pages/get-data/compustat-capital-iq-standard-poors/compustat/north-america-daily/fundamentals-quarterly/

We'll extract quarterly financial statement data and derive commonly used metrics if not available directly.

Variable references (for the quarterly reporting period, in USD):
- conm: company name
- tic: company ticker symbol
- rdq: report date of quarterly earnings
- revtq: total revenue 
- cogsq: cost of goods sold
- oiadpq: operating income after depreciation and amortisation
- dlcq: short-term (current) debt
- dlttq: long-term debt
- cheq: cash and cash equivalents at reporting point in time

The variables above will be used to calculated the following metrics:

- Gross Margin = (revtq – cogsq) / revtq 
    - "revtq" represents total revenues and "cogsq" represents the Cost of Goods Sold both at quarter level. The difference equals gross profit.
- EBITDA = oiadpq + dpq
    - Earnings Before Interest, Tax, Depreciation and Amortization. Since oiadpq already deducts depreciation and amortisation, adding dpq back returns EBITDA.
- Net Debt = (dlcq + dlttq) – che
    - Net Debt measures a company’s overall debt situation by offsetting its total debt with its liquid assets.


#### a) Download data and save it as a .csv so we avoiding repeated long queries to WRDS in case we clear memory. 
If the data has already been downloaded do not run this and skip to b)

In [ ]:
# Define the directory to download into
directory = 'data/wrds_company_fundamentals_data'
output_file = os.path.join(directory, 'wrds_company_fundamentals_data.csv')

# Check if the collated file already exists and delete it
if os.path.exists(output_file):
    os.remove(output_file)

# Pass parameters to a sql statement
params = {
    "tickers": tuple(tickers),
    "start_date": earliest_date,
    "end_date": latest_date
}

# Query WRDS to fetch data
quartely_company_fundamentals_df = db.raw_sql(
    "SELECT rdq as date, tic as ticker, revtq as revenue, ceqq as book_value, niq as net_income, (revtq - cogsq) / NULLIF(revtq, 0) as gross_margin, (revtq - cogsq) / NULLIF(atq, 0) as gross_profitability, "
    "oiadpq / NULLIF(atq, 0) as operating_profitability, (dlcq + dlttq) / NULLIF(atq, 0) as leverage, dlcq + dlttq - cheq as net_debt, oiadpq + dpq as ebitda, "
    "(dlcq + dlttq - cheq) / NULLIF((oiadpq + dpq), 0) as netdebt_to_ebitda " 
    "FROM comp_na_daily_current.fundq " 
    "WHERE tic in %(tickers)s and rdq BETWEEN %(start_date)s AND %(end_date)s ",
    params=params
)
#  revtq as rev, cogsq as cogs, oiadpq as op_income, dlcq as st_debt, dlttq as lt_debt, cheq as cash_eq, atq as total_assets, niq as net_income,  - base fields. Keeping only calculated metrics.

# Save the DataFrame to the directory
quartely_company_fundamentals_df.to_csv(output_file, index=False)

#### b) Open .csv file to memory

In [ ]:
# Define the directory containing the CSV files
directory = 'data/wrds_company_fundamentals_data'

# Open the file in a DataFrame for viewing
quartely_company_fundamentals_file = os.path.join(directory, 'wrds_company_fundamentals_data.csv')
quartely_company_fundamentals_df = pd.read_csv(quartely_company_fundamentals_file)

Extract unique downloaded tickers and compare to unique ticker list derived so far. Delete unmatched tickers from previous datasets.

In [27]:
# Extract unique tickers from WRDS download
quartely_company_fundamentals_tickers = set(quartely_company_fundamentals_df['ticker'].unique().tolist())
print(f"{len(quartely_company_fundamentals_tickers)} unique tickers found in WRDS quarterly company fundamentals data")

# Find missing tickers (tickers in short_interest_df but NOT in quartely_company_fundamentals_df)
print(f"{len(tickers)} unique tickers found in FINRA short interest data")
missing_tickers = set(tickers) - quartely_company_fundamentals_tickers

print(f"{len(missing_tickers)} tickers missing from the WRDS quarterly company fundamentals compared to the short interest file")

# Remove records with missing tickers from short_interest_df and daily stock data
short_interest_df = short_interest_df[~short_interest_df['symbolCode'].isin(missing_tickers)]

# Update ticker variable
tickers = short_interest_df['symbolCode'].unique().tolist()
print(f"Updated ticker list. New unique ticker count is: {len(tickers)}")

6086 unique tickers found in WRDS quarterly company fundamentals data
16857 unique tickers found in FINRA short interest data
10771 tickers missing from the WRDS quarterly company fundamentals compared to the short interest file
Updated ticker list. New unique ticker count is: 6086


#### ii) Stock Data (Daily Level) - Prices and Volume
https://wrds-www.wharton.upenn.edu/pages/get-data/compustat-capital-iq-standard-poors/compustat/north-america-daily/security-daily/

Variable Reference:
- conm: company name
- datadate: record date
- tic: ticker symbol
- cshoc: shares outstanding
- cshtrd: trading Volume - daily
- prccd: price - close - daily

Filtering the data to fetch only USA stocks in order to be able to inspect the dataset sensibly without the need for currency conversions and consistency of financial statement data.

#### a) Download data and save it as a .csv so we avoiding repeated long queries to WRDS in case we clear memory. 
If the data has already been downloaded do not run this and skip to b)

In [ ]:
# Define the directory to download into
directory = 'data/wrds_stock_daily_data'
output_file = os.path.join(directory, 'wrds_stock_daily_data.csv')

# Check if the collated file already exists and delete it
if os.path.exists(output_file):
    os.remove(output_file)

# Pass parameters to a sql statement
params = {
    "tickers": tuple(tickers),
    "start_date": earliest_date,
    "end_date": latest_date
}

# Query WRDS to fetch data
daily_stock_data_df = db.raw_sql(
    "SELECT datadate, conm as company_name, tic as ticker, prccd as price_close, cshtrd as volume, cshoc as shares_outstanding, prccd * cshoc as market_cap, eps " 
    "FROM comp_na_daily_all.secd " 
    "WHERE tic in %(tickers)s and datadate BETWEEN %(start_date)s AND %(end_date)s AND fic = 'USA'",
    params=params
)

# Save the DataFrame to the directory
daily_stock_data_df.to_csv(output_file, index=False)

#### b) Open .csv file to memory

In [28]:
# Define the directory containing the CSV files
directory = 'data/wrds_stock_daily_data'

# Open the file in a DataFrame for viewing
daily_stock_data_file = os.path.join(directory, 'wrds_stock_daily_data.csv')
daily_stock_data_df = pd.read_csv(daily_stock_data_file)

Extract unique downloaded tickers and compare to short interest file dataset. Delete unmatched tickers from previous datasets.

In [29]:
# Extract unique tickers from WRDS download
daily_stock_data_tickers = set(daily_stock_data_df['ticker'].unique().tolist())
print(f"{len(daily_stock_data_tickers)} unique tickers found in WRDS daily stock data")

# Find missing tickers (tickers in ticker list but NOT in daily_stock_data_df)
print(f"{len(tickers)} unique tickers found in FINRA short interest data")
missing_tickers = set(tickers) - daily_stock_data_tickers

print(f"{len(missing_tickers)} tickers missing from the WRDS daily stock data compared to the short interest file")

# Remove records with missing tickers from short_interest_df and quartely_company_fundamentals_df
short_interest_df = short_interest_df[~short_interest_df['symbolCode'].isin(missing_tickers)]
quartely_company_fundamentals_df = quartely_company_fundamentals_df[~quartely_company_fundamentals_df['ticker'].isin(missing_tickers)]

# Update ticker variable
tickers = short_interest_df['symbolCode'].unique().tolist()
print(f"Updated ticker list. New unique ticker count is: {len(tickers)}")

4707 unique tickers found in WRDS daily stock data
6086 unique tickers found in FINRA short interest data
1379 tickers missing from the WRDS daily stock data compared to the short interest file
Updated ticker list. New unique ticker count is: 4707


### 3) Yahoo Finance Data

##### a) Market Data - S&P500 and VIX

In [ ]:
# Define the directory containing the CSV files
directory = 'data/yahoo_finance_sp500_vix_data'
output_file = os.path.join(directory, 'yahoo_finance_sp500_vix_data.csv')

# Check if the collated file already exists and delete it
if os.path.exists(output_file):
    os.remove(output_file)

# Define ticker symbols
tickers = ["^GSPC", "^VIX"]  # S&P 500 and VIX

# Fetch data (Closing Prices and Volume)
sp500_vix_data_df = yf.download(tickers, start=earliest_date, end=latest_date, progress=False)[['Close']]

# Flatten the MultiIndex to standard column names
sp500_vix_data_df.columns = [f"{col[0]}_{col[1]}" for col in sp500_vix_data_df.columns]

# Rename columns to match the requested format
sp500_vix_data_df = sp500_vix_data_df.rename(columns={
    'Close_^GSPC': 'sp500_price_close',
    'Close_^VIX': 'vix_price_close'
})

# Reorder columns
sp500_vix_data_df = sp500_vix_data_df[['sp500_price_close', 'vix_price_close']]

# Format date index to YYYY-MM-DD and reset index
sp500_vix_data_df.index = sp500_vix_data_df.index.strftime('%Y-%m-%d')
sp500_vix_data_df = sp500_vix_data_df.reset_index()

# Rename the date column to 'date'
sp500_vix_data_df = sp500_vix_data_df.rename(columns={'index': 'date'})

# Save the collated DataFrame to the same directory
sp500_vix_data_df.to_csv(output_file, index=False)


#### b) Open .csv file to memory

In [30]:
# Define the directory containing the CSV files
directory = 'data/yahoo_finance_sp500_vix_data'

# Open the collated file in a DataFrame for viewing
sp500_vix_data_file = os.path.join(directory, 'yahoo_finance_sp500_vix_data.csv')
sp500_vix_data_df = pd.read_csv(sp500_vix_data_file)

### 4) Join datasets, perform data cleansing and compute additional metrics

a) Join datasets and foward fill datapoints where relevant

In [185]:
# Renaming columns used as join criteria between datasets
daily_stock_data_df.rename(columns={"datadate": "date"}, inplace=True)
sp500_vix_data_df.rename(columns={"Date": "date"}, inplace=True)
short_interest_df.rename(columns={"settlementDate": "date"}, inplace=True)
short_interest_df.rename(columns={"symbolCode": "ticker"}, inplace=True)

# Merge stock data and index data on date
merged_df = pd.merge(daily_stock_data_df, sp500_vix_data_df, on="date", how="left")

# Merge ticker-level data (on date + ticker)
merged_df = pd.merge(merged_df, quartely_company_fundamentals_df, on=["date", "ticker"], how="left")
merged_df = pd.merge(merged_df, short_interest_df, on=["date", "ticker"], how="left")

# Sort by date and ticker
merged_df = merged_df.sort_values(by=["date", "ticker"]).reset_index(drop=True)

# Forward fill ticker-specific, point-in-time values (e.g. short interest, quarterly gross profit, etc.)
ticker_cols = list(merged_df.columns)
ticker_cols.remove("date")  # Exclude date column from filling
ticker_cols.remove("ticker")  # Exclude ticker column from filling
merged_df[ticker_cols] = merged_df.groupby("ticker")[ticker_cols].ffill()

b) Cleanse dataset and improve readability

In [186]:
# Scale relevant fields from short interest file and daily stock data to millions in order to match the fundamentals file
divisor = 1_000_000
cols_to_divide = ['volume', 'shares_outstanding', 'market_cap', 'short_volume', 'avg_daily_volume']

merged_df[cols_to_divide] = merged_df[cols_to_divide] / divisor

# Identify and remove tickers with 0 shares outstanding.
tickers_zero_shares = merged_df.loc[merged_df['shares_outstanding'] == 0, 'ticker'].unique()
print("Count of tickers with 0 shares outstanding:", len(tickers_zero_shares))

merged_df = merged_df[~merged_df['ticker'].isin(tickers_zero_shares)].copy()
print("DataFrame shape after removing tickers with 0 shares outstanding:", merged_df.shape)

# Identify and remove tickers where market capitalisation has been below $100,000,000.
tickers_low_market_cap = merged_df.loc[merged_df['market_cap'] < 100, 'ticker'].unique()
print("Count of tickers where market cap has been below $100M:", len(tickers_low_market_cap))

merged_df = merged_df[~merged_df['ticker'].isin(tickers_low_market_cap)].copy()
print("DataFrame shape after removing tickers with low market cap:", merged_df.shape)

# Identify and remove tickers where quarterly revenue been below $100,000.
tickers_low_revenue = merged_df.loc[merged_df['revenue'] < 0.1, 'ticker'].unique() 
print("Count of tickers with where quarterly revenue has been below $100K:", len(tickers_low_revenue))

merged_df = merged_df[~merged_df['ticker'].isin(tickers_low_revenue)].copy()
print("DataFrame shape after removing tickers with low revenue:", merged_df.shape)

# Final count of tickers
tickers_final = merged_df['ticker'].unique().tolist()
print(f"Final count of tickers is: {len(tickers_final)}")

# Switch column order for better readability
new_order = ['date', 'ticker', 'company_name', 'price_close', 'shares_outstanding', 'market_cap', 'volume', 'short_volume', 'avg_daily_volume', 'days_to_cover',
             'sp500_price_close', 'vix_price_close', 'eps', 'book_value', 'revenue', 'gross_margin', 'gross_profitability', 'ebitda', 'operating_profitability', 
             'net_income', 'net_debt', 'netdebt_to_ebitda', 'leverage'
            ]
																							
merged_df = merged_df[new_order]

Count of tickers with 0 shares outstanding: 1
DataFrame shape after removing tickers with 0 shares outstanding: (3800015, 23)
Count of tickers where market cap has been below $100M: 1609
DataFrame shape after removing tickers with low market cap: (2495330, 23)
Count of tickers with where quarterly revenue has been below $100K: 206
DataFrame shape after removing tickers with low revenue: (2357504, 23)
Final count of tickers is: 2891


c) Adjust for possible stock splits

In [187]:
# Make sure the DataFrame is sorted by ticker and date.
merged_df = merged_df.sort_values(['ticker', 'date'])

# Step 1: Compute the previous day's shares outstanding within each ticker.
merged_df['prev_shares'] = merged_df.groupby('ticker')['shares_outstanding'].shift(1)
# For the first observation of each ticker, fill missing value with the current day's shares.
merged_df['prev_shares'] = merged_df['prev_shares'].fillna(merged_df['shares_outstanding'])

# Step 2: Compute the day-to-day ratio of shares outstanding.
merged_df['shares_ratio'] = merged_df['shares_outstanding'] / merged_df['prev_shares']

# Step 3: Define a threshold to detect a split.
# If the shares_ratio is greater than or equal to the threshold (e.g., 1.5), assume a split occurred.
split_threshold = 1.5

# For days when a split is detected, compute a multiplier as 1 / shares_ratio.
# Otherwise, the multiplier is 1.
merged_df['split_multiplier'] = np.where(
    (merged_df['shares_ratio'] >= split_threshold).fillna(False),
    1 / merged_df['shares_ratio'],
    1.0
)

# Step 4: Shift the split multiplier backward (i.e. forward in time)
# so that the multiplier detected on the split day is applied to the previous day.
# This means that the day on which the split is reported is treated as the new baseline.
merged_df['split_multiplier_shifted'] = merged_df.groupby('ticker')['split_multiplier'].shift(-1).fillna(1.0)

# Step 5: Compute the reverse cumulative product of the shifted multiplier within each ticker.
def reverse_cumprod(series):
    return series.iloc[::-1].cumprod().iloc[::-1]

merged_df['adjustment_factor'] = merged_df.groupby('ticker')['split_multiplier_shifted'].transform(reverse_cumprod)

# Step 6: Compute the split-adjusted price.
merged_df['price_close_adj'] = merged_df['price_close'] * merged_df['adjustment_factor']

# Optional: Clean up the temporary columns.
# merged_df.drop(columns=['prev_shares', 'shares_ratio', 'split_multiplier', 'split_multiplier_shifted', 'adjustment_factor'], inplace=True)



d) Compute additional metrics

In [188]:
# ==========================================
# 1. Fundamental Ratios
# ==========================================

# Price-to-Earnings (P/E) Ratio: Only compute if EPS > 0; otherwise, set to NaN.
merged_df['pe_ratio'] = np.where((merged_df['eps'] > 0).fillna(False),
                                 merged_df['price_close'] / merged_df['eps'], # can use non-adjusted price_close as eps is as reported at that point in time
                                 np.nan)

# Price-to-Book (P/B) Ratio: Only compute if book_value > 0; otherwise, set to NaN.
merged_df['pb_ratio'] = np.where((merged_df['book_value'] > 0).fillna(False),
                                 merged_df['market_cap'] / merged_df['book_value'],
                                 np.nan)

# Price-to-Sales (P/S) Ratio: Only compute if revenue > 0; otherwise, set to NaN.
merged_df['ps_ratio'] = np.where((merged_df['revenue'] > 0).fillna(False),
                                 merged_df['market_cap'] / merged_df['revenue'],
                                 np.nan)

# Price-to-EBITDA (P/EBITDA) Ratio: Only compute if EBITDA > 0; otherwise, set to NaN.
merged_df['pebitda_ratio'] = np.where((merged_df['ebitda'] > 0).fillna(False),
                                      merged_df['market_cap'] / merged_df['ebitda'],
                                      np.nan)


# Enterprise Value (EV) and EV/EBITDA:

# Compute EV as market_cap + net_debt.
merged_df['enterprise_value'] = merged_df['market_cap'] + merged_df['net_debt']

# Only compute EV/EBITDA if EBITDA > 0.
merged_df['ev_ebitda'] = np.where((merged_df['ebitda'] > 0).fillna(False),
                                  merged_df['enterprise_value'] / merged_df['ebitda'],
                                  np.nan)


# Return on Equity (ROE): net_income / book_value
merged_df['roe'] = merged_df['net_income'] / merged_df['book_value'].replace({0: np.nan})

# Net Margin: net_income / revenue
merged_df['net_margin'] = merged_df['net_income'] / merged_df['revenue'].replace({0: np.nan})

# ==========================================
# 2. Growth Metrics
# ==========================================

def compute_quarterly_growth(series):
    """
    Computes quarterly growth for a forward-filled series.
    The function calculates the percentage change only on days when the value changes
    (i.e., a new quarterly report is available), and then forward fills that value
    until the next change.
    """
    # Create a boolean mask: True when the reported value is different from the previous day.
    mask = series != series.shift(1)
    
    # Compute the percentage change (growth) on a day-by-day basis.
    growth = series.pct_change(fill_method=None)
    
    # Only keep the computed growth on the change days; elsewhere, set to NaN.
    growth = growth.where(mask)
    
    # Forward fill the computed growth so that every day between reports has the same growth value.
    growth = growth.ffill()
    
    return growth

# Compute quarterly growth for EPS and revenue by grouping on ticker.
merged_df['eps_growth'] = merged_df.groupby('ticker')['eps'].transform(compute_quarterly_growth)
merged_df['revenue_growth'] = merged_df.groupby('ticker')['revenue'].transform(compute_quarterly_growth)

# ==========================================
# 3. Volatility Metrics
# ==========================================

# Compute daily returns from price_close with fill_method=None to avoid the warning.
merged_df['daily_return'] = merged_df['price_close_adj'].pct_change(fill_method=None)

# 30-day rolling volatility (standard deviation of daily returns)
merged_df['volatility_30d'] = merged_df['daily_return'].rolling(window=30, min_periods=15).std()

# 90-day rolling volatility
merged_df['volatility_90d'] = merged_df['daily_return'].rolling(window=90, min_periods=45).std()

# ==========================================
# 4. Momentum Metrics
# ==========================================

# 10 day momentum: cumulative return over the past 10 trading days.
merged_df['momentum_10d'] = merged_df['price_close_adj'] / merged_df['price_close_adj'].shift(10) - 1

# 20 day momentum: cumulative return over the past 20 trading days.
merged_df['momentum_20d'] = merged_df['price_close_adj'] / merged_df['price_close_adj'].shift(20) - 1

# 50 day momentum: cumulative return over the past 50 trading days.
merged_df['momentum_50d'] = merged_df['price_close_adj'] / merged_df['price_close_adj'].shift(50) - 1

# 100 day momentum: cumulative return over the past 100 trading days.
merged_df['momentum_100d'] = merged_df['price_close_adj'] / merged_df['price_close_adj'].shift(100) - 1

# 200 day momentum: cumulative return over the past 200 trading days.
merged_df['momentum_200d'] = merged_df['price_close_adj'] / merged_df['price_close_adj'].shift(200) - 1

In [ ]:
# ==========================================
# Display a sample of the resulting DataFrame with new factors and check split adjustment
# ==========================================

selected_columns = ['date', 'ticker', 'price_close', 'price_close_adj', 'eps', 'pe_ratio', 'pb_ratio', 'ps_ratio', 'pebitda_ratio', 
                    'ev_ebitda', 'roe', 'net_margin', 'eps_growth', 'revenue_growth', 
                    'volatility_30d', 'volatility_90d', 'momentum_10d', 'momentum_50d', 'momentum_200d', 'shares_outstanding', 'prev_shares', 'shares_ratio', 'split_multiplier', 'split_multiplier_shifted', 'adjustment_factor']

merged_df.loc[merged_df['ticker'] == 'SMCI', selected_columns].sort_values(by=["date"]).tail(75) #use SMCI or NVIDIA as they've had recent splits


,date,ticker,price_close,price_close_adj,eps,pe_ratio,pb_ratio,ps_ratio,pebitda_ratio,ev_ebitda,roe,net_margin,eps_growth,revenue_growth,volatility_30d,volatility_90d,momentum_10d,momentum_50d,momentum_200d,shares_outstanding,prev_shares,shares_ratio,split_multiplier,split_multiplier_shifted,adjustment_factor
3495549,2024-09-27,SMCI,419.74,41.97,19.35,21.69,4.49,4.63,63.23,64.53,0.06,0.07,0.41,0.38,0.05,0.05,-0.08,-0.48,0.60,58.56,58.56,1.00,1.00,1.00,0.10
3499669,2024-09-30,SMCI,416.40,41.64,19.35,21.52,4.46,4.59,62.73,64.03,0.06,0.07,0.41,0.38,0.05,0.05,-0.07,-0.48,0.57,58.56,58.56,1.00,1.00,0.10,0.10
3503787,2024-10-01,SMCI,40.55,40.55,19.35,2.10,4.34,4.47,61.09,62.38,0.06,0.07,0.41,0.38,0.05,0.05,-0.08,-0.48,0.48,585.57,58.56,10.00,0.10,1.00,1.00
3507904,2024-10-02,SMCI,42.00,42.00,19.35,2.17,4.50,4.63,63.27,64.57,0.06,0.07,0.41,0.38,0.05,0.05,-0.04,-0.46,0.43,585.57,585.57,1.00,1.00,1.00,1.00
3512024,2024-10-03,SMCI,41.55,41.55,19.35,2.15,4.45,4.58,62.59,63.89,0.06,0.07,0.41,0.38,0.05,0.05,-0.05,-0.42,0.38,585.57,585.57,1.00,1.00,1.00,1.00
3516144,2024-10-04,SMCI,41.23,41.23,19.35,2.13,4.41,4.55,62.11,63.41,0.06,0.07,0.41,0.38,0.05,0.05,-0.10,-0.41,0.28,585.57,585.57,1.00,1.00,1.00,1.00
3520267,2024-10-07,SMCI,47.74,47.74,19.35,2.47,5.11,5.27,71.92,73.21,0.06,0.07,0.41,0.38,0.06,0.05,0.02,-0.33,0.51,585.57,585.57,1.00,1.00,1.00,1.00
3524381,2024-10-08,SMCI,45.35,45.35,19.35,2.34,4.86,5.00,68.32,69.61,0.06,0.07,0.41,0.38,0.06,0.05,-0.02,-0.35,0.50,585.57,585.57,1.00,1.00,1.00,1.00
3528502,2024-10-09,SMCI,47.29,47.29,19.35,2.44,5.06,5.22,71.24,72.54,0.06,0.07,0.41,0.38,0.06,0.05,0.03,-0.29,0.55,585.57,585.57,1.00,1.00,1.00,1.00
3532616,2024-10-10,SMCI,46.47,46.47,19.35,2.40,4.98,5.13,70.00,71.30,0.06,0.07,0.41,0.38,0.05,0.05,0.15,-0.34,0.60,585.57,585.57,1.00,1.00,1.00,1.00


In [84]:
# Assume merged_df is already loaded.
# For example:
# merged_df = pd.read_csv('your_data.csv')

# Specify the field (column) you're interested in.
field_name = 'netdebt_to_ebitda'  # Replace with your actual column name

# Calculate the 99th percentile threshold for the chosen field.
threshold = merged_df[field_name].quantile(0.01)
print(f"99th percentile threshold for '{field_name}': {threshold}")

# Extract rows where the field's value is greater than or equal to the threshold.
filtered_df = merged_df[merged_df[field_name] < threshold]

# Sort the filtered DataFrame in descending order by the specified field.
sorted_filtered_df = filtered_df.sort_values(by=field_name, ascending=False)

# Display the full sorted DataFrame.
print("Filtered and sorted DataFrame:")
sorted_filtered_df.tail(10)


99th percentile threshold for 'netdebt_to_ebitda': -117.73723431281226
Filtered and sorted DataFrame:


,date,company_name,ticker,price_close,volume,shares_outstanding,market_cap,eps,sp500_price_close,vix_price_close,revenue,book_value,net_income,gross_margin,gross_profitability,operating_profitability,leverage,net_debt,ebitda,netdebt_to_ebitda,short_volume,avg_daily_volume,days_to_cover
3791161,2024-10-09,LIFE360 INC,LIF,42.07,0.10,74.14,"3,118.86",NaN,"5,792.04",20.86,84.86,326.72,-10.96,0.77,0.16,-0.01,0.00,-159.90,0.00,"-53,300.00",0.35,0.30,1.18
3791135,2024-09-03,LIFE360 INC,LIF,37.25,0.23,74.14,"2,761.53",NaN,"5,528.93",20.72,84.86,326.72,-10.96,0.77,0.16,-0.01,0.00,-159.90,0.00,"-53,300.00",0.18,0.33,1.00
3791160,2024-10-08,LIFE360 INC,LIF,40.95,0.15,74.14,"3,035.83",NaN,"5,751.13",21.42,84.86,326.72,-10.96,0.77,0.16,-0.01,0.00,-159.90,0.00,"-53,300.00",0.35,0.30,1.18
3791159,2024-10-07,LIFE360 INC,LIF,41.58,0.23,74.14,"3,082.53",NaN,"5,695.94",22.64,84.86,326.72,-10.96,0.77,0.16,-0.01,0.00,-159.90,0.00,"-53,300.00",0.35,0.30,1.18
3791138,2024-09-06,LIFE360 INC,LIF,37.50,0.36,74.14,"2,780.06",NaN,"5,408.42",22.38,84.86,326.72,-10.96,0.77,0.16,-0.01,0.00,-159.90,0.00,"-53,300.00",0.18,0.33,1.00
3791158,2024-10-04,LIFE360 INC,LIF,40.02,0.21,74.14,"2,966.88",NaN,"5,751.07",19.21,84.86,326.72,-10.96,0.77,0.16,-0.01,0.00,-159.90,0.00,"-53,300.00",0.35,0.30,1.18
3791147,2024-09-19,LIFE360 INC,LIF,35.70,0.28,74.14,"2,646.62",NaN,"5,713.64",16.33,84.86,326.72,-10.96,0.77,0.16,-0.01,0.00,-159.90,0.00,"-53,300.00",0.17,0.37,1.00
3791134,2024-08-30,LIFE360 INC,LIF,39.43,0.23,74.14,"2,923.14",NaN,"5,648.40",15.00,84.86,326.72,-10.96,0.77,0.16,-0.01,0.00,-159.90,0.00,"-53,300.00",0.18,0.33,1.00
3791154,2024-09-30,LIFE360 INC,LIF,39.35,0.14,74.14,"2,917.21",NaN,"5,762.48",16.73,84.86,326.72,-10.96,0.77,0.16,-0.01,0.00,-159.90,0.00,"-53,300.00",0.35,0.30,1.18
3791151,2024-09-25,LIFE360 INC,LIF,39.05,0.18,74.14,"2,894.97",NaN,"5,722.26",15.41,84.86,326.72,-10.96,0.77,0.16,-0.01,0.00,-159.90,0.00,"-53,300.00",0.17,0.37,1.00
